In [1]:
import src.data as data

data.DataFetcher.main()
data.PreProcess.main(False)



Update Files
Sat Apr 13 17:39:12 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.12 Secs
Sat Apr 13 17:39:12 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.06 Secs
Sat Apr 13 17:39:12 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.02 Secs
Sat Apr 13 17:39:15 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 3.00 Secs
Sat Apr 13 17:39:21 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 5.91 Secs
Sat Apr 13 17:39:22 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240403.feather Done! Cost 0.14 Secs
Sat Apr 13 17:39:22 2024 : Updated ~ data\DataBase\DB_models\risk_exp\2024\risk_exp.20240408.feather Done! Cost 0.08 Secs
Sat Apr 13 17:39:22 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240408.feather Done! Cost 0.12 Secs
Sat Apr 13 17:39:22 2024 : Updated ~ data\DataBase\DB_models\risk_exp\2024\risk_exp.20240409.fea

24-04-13 17:40:30|MOD:PreProcess  |: if_train is False , Data Processing start!
24-04-13 17:40:30|MOD:PreProcess  |: 3 datas :['y', 'trade_day', 'trade_30m']


Sat Apr 13 17:40:30 2024 : All Updates Done! Cost 78.61 Secs
Sat Apr 13 17:40:30 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 0.61 secs
labels blocks reading ret20_lag DataBase's ...... cost 0.52 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_exp DataBase's ...... cost 1.99 secs
models blocks merging ...... cost 0.16 secs
y blocks process ...... cost 2.75 secs
y blocks masking ...... cost 0.11 secs
y blocks saving  ...... cost 0.12 secs
y blocks norming ...... cost 0.00 secs
Sat Apr 13 17:40:37 2024 : y finished! Cost 6.31 Seconds
Sat Apr 13 17:40:37 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 1.01 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.07 secs
trade_day blocks masking ...... cost 0.11 secs
trade_day blocks saving  ...... cost 0.18 secs
trade_day blocks norming ...... cost 0.00 secs
Sat Apr 13 17:40:38 2024 : trade_day finished! Cost 1.38 Seconds


24-04-13 17:40:44|MOD:PreProcess  |: Data Processing Finished! Cost 13.97 Seconds


... cost 1.13 secs
trade_30m blocks norming ...... cost 0.00 secs
Sat Apr 13 17:40:44 2024 : trade_30m finished! Cost 5.95 Seconds


In [ ]:
from run_model import ModelPred

model_preds = [
    ModelPred('gru_day'    , 'swalast' , 0 , 'gru_day_V0') ,
    ModelPred('gruRTN_day' , 'swalast' , 0 , 'gruRTN_day_V0') ,
    ModelPred('gruRES_day' , 'swalast' , 0 , 'gruRES_day_V0')
]
[md.get_df().deploy() for md in model_preds]

In [1]:


from run_model import ModelPred
md = ModelPred('gru_day' , 'swalast' , 0 , 'gru_day_V0')
md.get_df()

d:\Coding\learndl\learndl\model/gru_day/model_gru.yaml
Load  2 DataBlocks ...... cost 0.08 secs
Align 2 DataBlocks ...... cost 0.11 secs
20231201 new
Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]


ModelPred(model_name='gru_day', model_type='swalast', model_num=0, alias='gru_day_V0', df=       secid      date   gru_day
0          1  20240401  0.754627
1          2  20240401 -1.241937
2          4  20240401 -8.535885
3          6  20240401 -0.204401
4          7  20240401  1.486538
...      ...       ...       ...
5096  688799  20240403  1.810010
5097  688800  20240403  2.127807
5098  688819  20240403  2.988075
5099  688981  20240403  0.242482
5100  689009  20240403  0.015975

[15305 rows x 3 columns])